# Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
    
import time
import copy
import numpy as np

import matplotlib.pyplot as plt
from cycler import cycler
color_list  = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']
marker_list = ['o','x','*','v','^','d']
plt.rc('axes', prop_cycle=(cycler('color',color_list)))

from LRT import LRT
from LRT import moments
from LRT import figfuns

**Setup parameter struct:**

In [ ]:
par = LRT.ParStruct()
SAMPLE = '_p100'

**Seed:**

In [ ]:
np.random.seed(1917)

## Load data

In [ ]:
# a. load
data = LRT.dataStruct()
data.logY = np.transpose(np.genfromtxt(f'data/logY{SAMPLE}.csv',delimiter=','))

par.T, par.N = data.logY.shape
print(f'(T,N) = ({par.T},{par.N})')

# b. rescale
Y = np.exp(data.logY)
Y_lev = Y
Y = Y / np.mean(Y[0,:])
data.logY = np.log(Y)
    
# c. update par
par.simN = max([par.N,par.simN])
par.k = 5
par.k_lead = 5
par.depth = 10

## Attrition sample

In [ ]:
# a. setup
data_attr = LRT.dataStruct()
data_attr.logY = np.nan*np.empty(data.logY.shape)

# b. attrition
T0 = np.zeros((par.N,),dtype=int)
T1 = (par.T-1)*np.ones((par.N,),dtype=int)

ts = np.random.choice(range(par.T),size=(par.N,))
I = ts < (par.T/2)

T0[I] = ts[I]
T1[I == False] = ts[I == False]

for i in range(par.N): 
    ts = range(T0[i], T1[i]+1)
    data_attr.logY[ts,i] = data.logY[ts,i]

In [ ]:
share_selected = np.mean(np.isnan(data_attr.logY) == False)
print(f'selected {100.0*share_selected:.1f} percent of observations')

## Reduced sub-sample

In [ ]:
# a. setup
data_sub = LRT.dataStruct()
par_sub = copy.deepcopy(par)

# b. random sub-sample
I = np.random.binomial(1,share_selected,size=par.N) == 1
data_sub.logY = data.logY[:, I]

# c. update par
par_sub = copy.deepcopy(par)
par_sub.T, par_sub.N = data_sub.logY.shape

# Estimate

## Attribution sample

In [ ]:
model_sub = LRT.estimate(par_sub,data_sub,name='LRT (balanced)',color=color_list[0])
model_sub.marker = marker_list[0]
model_sub.name_short = 'LRT_balanced'

## Reduced sample

In [ ]:
model_attr = LRT.estimate(par,data_attr,name='LRT (attrition)',color=color_list[1])
model_attr.marker = marker_list[1]
model_attr.name_short = 'LRT_attrition'

## Simulate

In [ ]:
rng_state = np.random.get_state()

model_sub.data = LRT.simulate(model_sub.par,model_sub,data_sub,seed=None,rng_state=rng_state)
model_attr.data = LRT.simulate(model_attr.par,model_attr,data_attr,seed=None,rng_state=rng_state)

models = [model_sub, model_attr]

## Moments

For **data**:

In [ ]:
moments.calc_all(par,data,printprogress=True)
moments.calc_all(par,data_attr,printprogress=True)

For **models:**

In [ ]:
for model in models: 
    print(model.name)
    moments.calc_all(model.par,model.data,printprogress=True)

## Figures 

In [ ]:
prefix = 'attrition'

figfuns.age_profile(par, data, models, 0, 
                    varname_str='logY', prefix=prefix)

figfuns.age_profile(par, data, models, 1, 
                    varname_str='dlogY', prefix=prefix)

figfuns.age_profile(par, data, models, 5, 
                    varname_str='dlogY', prefix=prefix)

figfuns.heterogenous(par, data, models,
                     'dlogY', 0, 'life-time growth (demeaned)',
                     prefix=prefix, bounds=(-1.5,1.5))

figfuns.heterogenous(par, data, models,
                     'autocorr', 1, 'first-order autocorr.',
                     prefix=prefix, bounds=(-1,1))

figfuns.heterogenous(par, data, models,
                     'autocorr_level', 1, 'first-order autocorr. (levels)',
                     prefix=prefix, bounds=(-.25,1))

figfuns.heterogenous(par, data, models,
                     'std_dlogY', 1, 'std. of income growth',
                     prefix=prefix, bounds=(0,0.5))